In [1]:
from pynq import Overlay
from pynq.lib.video import *
import time

overlay = Overlay("/home/xilinx/base2.bit")

In [2]:
imgen_vdma = overlay.video.axi_vdma_0.readchannel
videoMode = common.VideoMode(640, 480, 24)
imgen_vdma.mode = videoMode
imgen_vdma.start()

hdmi_out = overlay.video.hdmi_out
hdmi_out._vdma = overlay.video.axi_vdma #Use the correct VDMA!
hdmi_out.configure(videoMode)
hdmi_out.start()

pixgen = overlay.pixel_generator_0

In [2]:
import socket
import pickle

# Initialize TCP client
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(('127.0.0.1', 9999))
print("Connected to server.")

done = 0
BRAM_A_WE = 0x0000 # pixgen.register_map.BRAM_A_WE  # WRITE ENABLE FLAG

PAUSE = 0x0001 #pixgen.register_map.PAUSE = 0x0001  # PAUSE FLAG
while True:
    if (not done):
        num_chunks = pickle.loads(client_socket.recv(4096))
        data = b""
        for i in range(num_chunks):
            chunk = client_socket.recv(4096)
            data += chunk                
        matrix=pickle.loads(data)
       
        for item in matrix:
            counterd = 0
            for data in item:
                if data:
                    item[counterd] = 1
                else:
                    item[counterd] = 0
                counterd = counterd + 1
            binary_str = ''.join(map(str, item))
            while len(binary_str) % 8 != 0:
                binary_str = '0' + binary_str
            for i in range(40):
                hex_int = int(binary_str[32*i:((i+1)*32)-1], 2)
                register_name = f"gp{i}"
                #setattr(pixgen.register_map, register_name, hex_int)
        done=1
    pixgen.register_map.gp40 = 0x0001 # done writing
    message = pickle.loads(client_socket.recv(2048))
    if message:
        pixgen.register_map.PAUSE = 0x0001
    else:
        pixgen.register_map.PAUSE = 0x0000

    frame = imgen_vdma.readframe() # Output next frame
    hdmi_out.writeframe(frame)

TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond